In [1]:
!pip install transformers datasets

In [2]:
!pip install peft

In [3]:
pip install trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.5/465.5 kB 19.2 MB/s eta 0:00:00


In [4]:
!pip install torch

In [5]:
from transformers import AutoTokenizer,AutoModelForCausalLM,TrainingArguments
from peft import LoraConfig,get_peft_model,prepare_model_for_kbit_training
from datasets import load_dataset
from trl import SFTTrainer # supervisied fine  tuning , for instruction fine tuning

In [6]:
import torch

model_name = "microsoft/DialoGPT-medium"
tokenizer = AutoTokenizer.from_pretrained(model_name)

tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(model_name,torch_dtype=torch.bfloat16,device_map="auto")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


pytorch_model.bin:   0%|          | 0.00/863M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/863M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [7]:
dataset = load_dataset("imdb")

README.md: 0.00B [00:00, ?B/s]

plain_text/train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

plain_text/test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

plain_text/unsupervised-00000-of-00001.p(…):   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [8]:
def tonkize_function(examples):
  return tokenizer(
      examples["text"],
      padding="max_length",
      truncation=True,
      max_length=512,
  )

In [9]:
tokenized_dataset = dataset.map(tonkize_function)

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [10]:
model = prepare_model_for_kbit_training(model)

lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["c_attn","c_proj",]
)


model = get_peft_model(model,lora_config)
model.print_trainable_parameters()


trainable params: 4,325,376 || all params: 359,148,544 || trainable%: 1.2043


/usr/local/lib/python3.12/dist-packages/peft/tuners/lora/layer.py:2285: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


In [ ]:
training_args = TrainingArguments(
    output_dir="./lora-dialogpt",
    num_train_epochs=1,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    warmup_steps=100,
    max_steps=1000,
    learning_rate=2e-4,
    fp16=True,
    logging_steps=25,
    save_strategy="steps",
    save_steps=500,
)


trainer = SFTTrainer(
    model =model,
    train_dataset=tokenized_dataset["train"],
    args=training_args,
    processing_class=tokenizer,
)



trainer.train()

model.save_pretrained("./lora-adapter")
eval_results = trainer.evaluate()
print(f"Evaluation results: {eval_results}")

The model is already on multiple devices. Skipping the move to device specified in `args`.


Step,Training Loss
25,2.292800
50,2.353800
75,2.189300
100,2.234200
125,2.251800
150,2.117500
175,2.052700
200,2.136300
225,2.098900
250,2.034200
